# FVCOM Grid Node Checker Demo
**Author: Jun Sasaki | Created: 2025-09-14 Updated: 2025-09-18**

**Purpose:** Visualize FVCOM grid nodes using xfvcom's core plotting functionality

This notebook demonstrates how to:
- Load FVCOM grid using FvcomInputLoader
- Use FvcomPlotter with FvcomPlotOptions for visualization
- Display node markers and numbers using make_node_marker_post
- Highlight specific nodes of interest
- Export node coordinate information

## Setup and Imports

In [ ]:
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
from cartopy.io.img_tiles import GoogleTiles, OSM

# Import xfvcom modules
from xfvcom import (
    FvcomInputLoader,
    FvcomPlotter,
    FvcomPlotConfig,
    FvcomPlotOptions,
    make_node_marker_post,
)

# Create output directory for saved figures
output_dir = Path("PNG")
output_dir.mkdir(exist_ok=True)

print("Setup complete!")

## 1. Define Grid File Path and Load Grid
Load the FVCOM grid file to get mesh structure and node coordinates

In [ ]:
# Define path to your grid file
grid_file = Path("~/Github/TB-FVCOM/goto2023/input/TokyoBay18_grd.dat").expanduser()

# UTM zone for Tokyo Bay (adjust for your region)
utm_zone = 54

# Check if file exists
if grid_file.exists():
    print(f"✓ Grid file found: {grid_file}")
    print(f"  File size: {grid_file.stat().st_size / 1024:.1f} KB")
else:
    print(f"✗ Grid file not found: {grid_file}")
    print("  Please update the path to your FVCOM grid file")

# Load grid using FvcomInputLoader
loader = FvcomInputLoader(
    grid_path=grid_file,
    utm_zone=utm_zone,
    add_dummy_time=False,  # We don't need dummy time for this
    add_dummy_siglay=False  # We don't need dummy sigma layers
)

# Get the dataset and grid object
grid_ds = loader.ds
grid_obj = loader.grid

print(f"\nGrid loaded successfully")
print(f"Number of nodes: {grid_obj.node}")
print(f"Number of elements: {grid_obj.nele}")
print(f"Coordinate range:")
print(f"  Longitude: {grid_ds.lon.min().values:.3f} - {grid_ds.lon.max().values:.3f}")
print(f"  Latitude: {grid_ds.lat.min().values:.3f} - {grid_ds.lat.max().values:.3f}")

## 2. Initialize Plotter
Create FvcomPlotter instance for visualization

In [ ]:
# Create plotter for visualization
cfg = FvcomPlotConfig()
plotter = FvcomPlotter(grid_ds, cfg)

print("FvcomPlotter initialized")

## 3. Display All Nodes with Markers
Show markers at all node positions using make_node_marker_post

In [ ]:
# Display all nodes with markers (no numbers for performance)
# Create array of all node indices (1-based for FVCOM convention)
all_nodes = np.arange(1, grid_obj.node + 1)

# Define marker styling for all nodes
marker_kwargs = {
    "marker": "o", 
    "color": "yellow", 
    "markersize": 1,  # Small size for all nodes
    "zorder": 4
}

# Create post-processing function for all nodes
pp_all_nodes = make_node_marker_post(
    all_nodes,
    plotter,
    marker_kwargs=marker_kwargs,
    text_kwargs=None,  # No text labels for performance
    index_base=1,
    respect_bounds=True,  # Only show nodes within view bounds
)

# Plot options
opts = FvcomPlotOptions(
    figsize=(12, 10),
    add_tiles=True,
    tile_provider=GoogleTiles(style="satellite"),
    mesh_color="yellow",
    mesh_linewidth=0.2,
    title="All Grid Nodes",
)

# Create the plot
ax = plotter.plot_2d(da=None, post_process_func=pp_all_nodes, opts=opts)

# Save figure
ax.figure.savefig(output_dir / "all_nodes_markers.png", dpi=300, bbox_inches='tight')
plt.show()
print("All nodes visualization saved to PNG/all_nodes_markers.png")

## 4. Display Specific Nodes with Numbers
Highlight and label specific nodes of interest

In [ ]:
# Define nodes of interest (one-based node numbers)
nodes_of_interest = [100, 200, 300, 500, 1000, 1500, 2000, 2500, 3000]

# Check if nodes are valid
valid_nodes = [n for n in nodes_of_interest if 1 <= n <= grid_obj.node]
invalid_nodes = [n for n in nodes_of_interest if n < 1 or n > grid_obj.node]

print(f"Valid nodes: {valid_nodes}")
if invalid_nodes:
    print(f"Invalid nodes (out of range): {invalid_nodes}")

# Define marker and text styling for specific nodes
mkw = {"marker": "o", "color": "red", "markersize": 5, "zorder": 5}
tkw = {"fontsize": 10, "color": "yellow", "ha": "center", "va": "bottom",
       "zorder": 6, "clip_on": True}

# Create post-processing function for specific nodes
pp_specific = make_node_marker_post(
    valid_nodes,
    plotter,
    marker_kwargs=mkw,
    text_kwargs=tkw,
    index_base=1,
    respect_bounds=False,  # Show all specified nodes
)

# Plot options with white background
opts = FvcomPlotOptions(
    figsize=(12, 10),
    add_tiles=True,
    tile_provider=GoogleTiles(style="satellite"),
    mesh_color="lightgray",
    mesh_linewidth=0.2,
    title="Specific Nodes Highlighted",
)

# Create the plot
ax = plotter.plot_2d(da=None, post_process_func=pp_specific, opts=opts)

# Save figure
ax.figure.savefig(output_dir / "specific_nodes.png", dpi=300, bbox_inches='tight')
plt.show()
print("Specific nodes visualization saved to PNG/specific_nodes.png")

## 5. Plot Nodes on Mesh Map (Zoomed View)
Similar to river input checker Section 5 - show nodes on zoomed map with satellite tiles

In [ ]:
# Set map domain for zoomed view (adjust these to your area of interest)
# Example: Focus on central Tokyo Bay
xlim = (139.72, 140.00)
ylim = (35.60, 35.67)

# Select subset of nodes for zoomed view
# You can define specific nodes or use a range
selected_nodes = [100, 200, 300, 500, 1000, 1500, 2000, 2500, 3000]

# Define marker and text styling
mkw = {"marker": "o", "color": "red", "markersize": 5, "zorder": 4}
tkw = {"fontsize": 10, "color": "yellow", "ha": "center", "va": "bottom",
       "zorder": 5, "clip_on": True}

# Create post-processing function
pp_zoomed = make_node_marker_post(
    selected_nodes,
    plotter,
    marker_kwargs=mkw,
    text_kwargs=tkw,
    index_base=1,
    respect_bounds=True,  # Only show nodes within xlim/ylim
)

# Plot options following river_input_checker style
opts = FvcomPlotOptions(
    figsize=(10, 12),
    add_tiles=True,
    tile_provider=GoogleTiles(style="satellite"),
    mesh_color="lightgray",
    mesh_linewidth=0.2,
    title="Node Locations on FVCOM Mesh (Zoomed View)",
    xlim=xlim,
    ylim=ylim,
)

# Create the plot
ax = plotter.plot_2d(da=None, post_process_func=pp_zoomed, opts=opts)

# Save figure
ax.figure.savefig(output_dir / "nodes_map_zoomed.png", dpi=300, bbox_inches='tight')
plt.show()

# Report which nodes are shown vs hidden
nodes_shown = []
nodes_hidden = []
for node_idx_1based in selected_nodes:
    node_idx_0based = node_idx_1based - 1
    if node_idx_0based < len(grid_ds.lon):
        lon = grid_ds.lon.values[node_idx_0based]
        lat = grid_ds.lat.values[node_idx_0based]
        if xlim[0] <= lon <= xlim[1] and ylim[0] <= lat <= ylim[1]:
            nodes_shown.append(node_idx_1based)
        else:
            nodes_hidden.append(node_idx_1based)

print(f"Nodes shown in this view: {nodes_shown}")
print(f"Nodes outside view bounds: {nodes_hidden}")
print("\nTo see all nodes regardless of bounds, set respect_bounds=False")

## 6. Combined Visualization
Show all nodes with specific nodes highlighted

In [ ]:
# Combine all nodes (small blue markers) with highlighted specific nodes (red with numbers)

# Define marker and text styling
mkw = {"marker": "o", "color": "red", "markersize": 4, "zorder": 5}
tkw = {"fontsize": 10, "color": "yellow", "ha": "left", "va": "bottom",
       "zorder": 6, "clip_on": True}
# Create a combined post-processing function
def combined_post_process(ax):
    # First, plot all nodes as small markers
    all_nodes_subset = np.arange(1, grid_obj.node + 1)
    pp_all = make_node_marker_post(
        all_nodes_subset,
        plotter,
        marker_kwargs={"marker": ".", "color": "lightgray", "markersize": 1, "zorder": 3},
        text_kwargs=None,
        index_base=1,
        respect_bounds=True,
    )
    pp_all(ax)
    
    # Then, highlight specific nodes with numbers
    highlight_nodes = [100, 500, 1000, 1500, 2000]
    pp_highlight = make_node_marker_post(
        highlight_nodes,
        plotter,
        marker_kwargs=mkw,
        text_kwargs=tkw,
        index_base=1,
        respect_bounds=True,
    )
    pp_highlight(ax)

# Plot options
opts = FvcomPlotOptions(
    figsize=(12, 10),
    add_tiles=True,
    tile_provider=GoogleTiles(style="satellite"),
    mesh_color="lightgray",
    mesh_linewidth=0.2,
    title="All Nodes with Highlighted Selection",
)

# Create the plot
ax = plotter.plot_2d(da=None, post_process_func=combined_post_process, opts=opts)

# Save figure
ax.figure.savefig(output_dir / "combined_view.png", dpi=300, bbox_inches='tight')
plt.show()
print("Combined visualization saved to PNG/combined_view.png")

## 7. Map View with All Nodes and Node Numbers

In [ ]:
# Create a full map view showing all nodes (or a subset for performance)
# Use every 50th node to avoid overcrowding

# Nodes to be plotted
display_nodes = np.arange(1, grid_obj.node + 1)

# Create post-processing function that shows ALL selected markers
pp_node_num = make_node_marker_post(
    display_nodes,
    plotter,
    marker_kwargs={"marker": "o", "color": "red", "markersize": 4, "zorder": 4},
    text_kwargs={"fontsize": 10, "color": "yellow", "ha": "left", "va": "bottom", "zorder": 5, "clip_on": True},
    index_base=1,
    respect_bounds=False,  # Show all markers regardless of xlim/ylim
)

# Full map extent
xlim_node_num = (float(grid_ds.lon.min()), float(grid_ds.lon.max()))
ylim_node_num = (float(grid_ds.lat.min()), float(grid_ds.lat.max()))
xlim_node_num = (139.85, 139.95)
ylim_node_num = (35.36, 35.45)


# Plot options for full view
opts_node_num = FvcomPlotOptions(
    figsize=(12, 10),
    add_tiles=True,
    tile_provider=GoogleTiles(style="satellite"),
    mesh_color="lightgray",
    mesh_linewidth=0.2,
    title=f"Node Locations on FVCOM Mesh",
    xlim=xlim_node_num,
    ylim=ylim_node_num,
)

# Create the full map plot
ax_node_num = plotter.plot_2d(da=None, post_process_func=pp_node_num, opts=opts_node_num)

# Save figure
ax_node_num.figure.savefig(output_dir / "nodes_num_map.png", dpi=300, bbox_inches='tight')
plt.show()
print(f"Map showing nodes and numbers; saved to PNG/nodes_num_map.png")
print(f"Total nodes displayed: {len(display_nodes)} out of {grid_obj.node}")